Training YOLO
===

### Load pre-trained YOLOv8 model

In [1]:
from ultralytics import YOLO

# load pretrained model
model = YOLO('yolov8n.pt')


### Make the YAML file

In [2]:
import os
os.getcwd()

'/home/pyml/git_vmirly/Udemy-ComputerVision/section4-object-detection'

In [3]:
%%writefile -a train.yaml

path: yolo_format/
train: images/train/
val: images/val/
test:

# Classes
names:
    0: person
    1: car
    2: bicycle
    3: motorcycle
    4: bus
    5: truck


Appending to train.yaml


### Training

In [ ]:
# diable wandb
os.environ['WANDB_DISABLED'] = 'true'

# run training
model.train(
    data='train.yaml',
    epochs=10,
    imgsz=640,
)

### Inference

  * load the trained model

In [2]:
# load the trained model
model = YOLO("runs/detect/train1/weights/best.pt")


#### Evluate on the testset

In [ ]:
import json
import pandas as pd

res_dir = "results/yolov8"
os.makedirs(res_dir, exist_ok=True)

df = pd.read_csv("data/data.csv")
df_test = df[df["tag"] == "test"]

for i, row in df_test.iterrows():
    img_path = row["imagepath"]
    results = model(img_path)
    boxes_xywhn = results[0].boxes.xywhn.cpu().numpy()
    labels = results[0].boxes.cls.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    filename = os.path.basename(img_path)

    res = {
        "image_path": img_path,
        "boxes_xywhn": boxes_xywhn.tolist(),
        "labels": labels.tolist(),
        "scores": scores.tolist(),
    }

    with open(os.path.join(res_dir, filename.replace(".jpg", ".json")), "wt") as f:
        json.dump(res, f)
    #break